<a href="https://colab.research.google.com/github/Charmiece/Final-Project---CreditCard-Fraud/blob/main/BalancedRandomForestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://fraudcreditcard.s3.amazonaws.com/clean_data_train.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("clean_data_train.csv"), sep=",", header=True)
# Show DataFrame
df.show()

+--------------------+-------------+------+------+----------+---------+------------------+--------+---+
|            merchant|     category|   amt|gender| unix_time|merch_lat|        merch_long|is_fraud|age|
+--------------------+-------------+------+------+----------+---------+------------------+--------+---+
|fraud_Rippin, Kub...|     misc_net|  4.97|     F|1325376018|36.011293|        -82.048315|       0| 34|
|fraud_Heller, Gut...|  grocery_pos|107.23|     F|1325376044|49.159047|       -118.186462|       0| 44|
|fraud_Lind-Buckridge|entertainment|220.11|     M|1325376051|43.150704|       -112.154481|       0| 60|
|fraud_Kutch, Herm...|gas_transport|  45.0|     M|1325376076|47.034331|       -112.561071|       0| 55|
| fraud_Keeling-Crist|     misc_pos| 41.96|     M|1325376186|38.674999|        -78.632459|       0| 36|
|fraud_Stroman, Hu...|gas_transport| 94.63|     F|1325376248|40.653382|-76.15266700000001|       0| 61|
|fraud_Rowe-Vander...|  grocery_net| 44.54|     F|1325376282|37.

In [ ]:
# importing the packages
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import date
import sklearn.exceptions
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.svm import OneClassSVM
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import IsolationForest
from imblearn.under_sampling import NearMiss 
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, metrics, preprocessing, svm
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report,accuracy_score
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
from imblearn.metrics import classification_report_imbalanced


from sklearn.ensemble import RandomForestClassifier

In [ ]:
data_train = pd.read_csv("https://fraudcreditcard.s3.amazonaws.com/clean_data_train.csv")

In [ ]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   merchant    1296675 non-null  object 
 1   category    1296675 non-null  object 
 2   amt         1296675 non-null  float64
 3   gender      1296675 non-null  object 
 4   unix_time   1296675 non-null  int64  
 5   merch_lat   1296675 non-null  float64
 6   merch_long  1296675 non-null  float64
 7   is_fraud    1296675 non-null  int64  
 8   age         1296675 non-null  int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 89.0+ MB


In [ ]:
print(data_train.shape)

(1296675, 9)


In [ ]:
# taking smaller sample to run the model faster

df_train= data_train.sample(frac = 0.1,random_state=1)
print(df_train.shape)
df_train.head(10)

(129668, 9)


,merchant,category,amt,gender,unix_time,merch_lat,merch_long,is_fraud,age
94102,fraud_Funk Group,grocery_net,20.00,F,1330158280,46.398331,-99.813959,0,74
198791,"fraud_Prosacco, Kreiger and Kovacek",home,284.88,F,1334260215,45.687331,-117.488135,0,79
1238587,"fraud_Langworth, Boehm and Gulgowski",shopping_net,5.07,F,1370037053,27.254081,-81.974799,0,44
619078,fraud_Conroy-Emard,food_dining,45.38,F,1348160812,34.551957,-83.374265,0,25
573850,fraud_Adams-Barrows,health_fitness,27.52,M,1346455370,43.032957,-83.521294,0,27
1201872,fraud_Effertz LLC,entertainment,131.75,F,1368814877,35.068173,-90.274482,0,45
61984,fraud_Morissette LLC,entertainment,146.69,F,1328478474,38.229734,-94.197224,0,35
552029,fraud_Wiegand-Lowe,grocery_net,74.27,M,1345809559,34.230048,-82.674878,0,80
1079425,fraud_Harris Group,food_dining,11.22,F,1364105244,33.700800,-80.342778,0,61
681734,fraud_Dach-Nader,misc_net,3.96,F,1350531664,29.905143,-98.185725,0,55


In [ ]:
df_train.isnull().sum()

merchant      0
category      0
amt           0
gender        0
unix_time     0
merch_lat     0
merch_long    0
is_fraud      0
age           0
dtype: int64

In [ ]:
# Create our features
X = pd.get_dummies(df_train, columns=['merchant', 'category', 'gender'])
                
# Create our target
y = df_train['is_fraud']

In [ ]:
X.describe()

,amt,unix_time,merch_lat,merch_long,is_fraud,age,merchant_fraud_Abbott-Rogahn,merchant_fraud_Abbott-Steuber,merchant_fraud_Abernathy and Sons,merchant_fraud_Abshire PLC,...,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_F,gender_M
count,129668.000000,1.296680e+05,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,...,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000,129668.000000
mean,70.061919,1.349218e+09,38.532677,-90.195662,0.005437,48.721990,0.001365,0.001504,0.001419,0.001473,...,0.095397,0.086135,0.049311,0.060840,0.070364,0.075238,0.089536,0.031272,0.549758,0.450242
std,153.627239,1.279134e+07,5.094736,13.758655,0.073535,17.374394,0.036921,0.038750,0.037643,0.038351,...,0.293764,0.280565,0.216516,0.239038,0.255761,0.263776,0.285517,0.174053,0.497520,0.497520
min,1.000000,1.325376e+09,19.035472,-166.657834,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.630000,1.338757e+09,34.751272,-96.899931,0.000000,35.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,47.360000,1.349244e+09,39.360173,-87.408320,0.000000,47.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,82.940000,1.359235e+09,41.938529,-80.190236,0.000000,60.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,17897.240000,1.371817e+09,66.645176,-66.963975,1.000000,98.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



Random Forest algorithm will sample the data and build several smaller, simpler decision trees.





* Are robust against overfitting as all of those weak learners are trained on different pieces of the data.
* Can be used to rank the importance of input variables in a natural way.
* Can handle thousands of input variables without variable deletion.
* Are robust to outliers and nonlinear data.
* Run efficiently on large datasets.

In [ ]:
# Import dependencies
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.ensemble import EasyEnsembleClassifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model = rf_model.fit(X,y)
print(rf_model)

BalancedRandomForestClassifier(random_state=1)


In [ ]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X)
balanced_accuracy_score(y, y_pred)

1.0

In [ ]:
# Display the confusion matrix
# Calculating the confusion matrix.
y_pred = rf_model.predict(X)
cm = confusion_matrix(y, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"],
    columns=["Predicted high_risk", "Predicted low_risk"]
)

# Displaying results
display(cm_df)

,Predicted high_risk,Predicted low_risk
Actual high_risk,128963,0
Actual low_risk,0,705


In [ ]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00    128963
          1       1.00      1.00      1.00      1.00      1.00      1.00       705

avg / total       1.00      1.00      1.00      1.00      1.00      1.00    129668



In [ ]:
# List the features sorted in descending order by feature importance
feature_importances = sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)
feature_importances

[(0.48614084829942045, 'is_fraud'),
 (0.2599269969463093, 'amt'),
 (0.021144147486957497, 'unix_time'),
 (0.020322827420363, 'age'),
 (0.01915213938993386, 'merch_long'),
 (0.01817743627613381, 'merch_lat'),
 (0.011712888421249784, 'category_shopping_net'),
 (0.008614232942487192, 'category_grocery_pos'),
 (0.007418190381723475, 'category_misc_net'),
 (0.005964893026386243, 'category_health_fitness'),
 (0.00551425762957958, 'category_gas_transport'),
 (0.0049217042637822814, 'category_kids_pets'),
 (0.0045021871629209115, 'category_home'),
 (0.004238441067298812, 'gender_F'),
 (0.004062988641344798, 'category_food_dining'),
 (0.0036991274358904947, 'gender_M'),
 (0.0032370571536038586, 'category_personal_care'),
 (0.0024714785287537917, 'category_misc_pos'),
 (0.0024128259736867605, 'category_grocery_net'),
 (0.0023209215681860053, 'category_travel'),
 (0.002042822987912366, 'category_entertainment'),
 (0.001815845026620902, 'category_shopping_pos'),
 (0.00098884703462885, 'merchant_fr